In [1]:
import pandas as pd
import seaborn as sb
import requests as req
from helper import *
from standardize import *
import geopandas as gpd

In [2]:
lead_df = pd.DataFrame(pd.read_json(soup.contents[0]).loc['records'].loc['result'])

In [3]:
lead_df.shape

(77191, 7)

In [4]:
lead_df.rename(columns={c:camel_to_snake(c) for c in lead_df.columns}, inplace=True)

In [5]:
crime_df = pd.DataFrame(pd.read_json(crime_soup.contents[0]).loc['records'].loc['result'])

In [6]:
crime_df.shape

(77191, 7)

In [7]:
crime_df.head()

,City,House Number Range,House Number contd.,State,Street Name,Zip Code,_id
0,MILWAUKEE,100,,WISCONSIN,E ABERT PL,53212,1
1,MILWAUKEE,940,,WISCONSIN,E ABERT PL,53212,2
2,MILWAUKEE,1301,,WISCONSIN,E ALBION ST,53202,3
3,MILWAUKEE,1309,11,WISCONSIN,E ALBION ST,53202,4
4,MILWAUKEE,1316,18,WISCONSIN,E ALBION ST,53202,5


In [ ]:
fp = 'jun18anogeo.shp'
map_df = gpd.read_file(fp)
# check data type so we can see that this is not a normal dataframe, but a GEOdataframe
map_df.head()